In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

C:\Users\Baha Tegar\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account

In [3]:
key_path = './service_account/gentle-keyword-423715-j0-03be08ad6412.json'

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/bigquery"]
)

In [4]:
from google.cloud import bigquery

client = bigquery.Client(
    credentials = credentials,
    project=credentials.project_id
)

In [7]:
df_pred = pd.read_csv('./pred-revenue-next90days.csv')
print(df_pred.info())

df_pred.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17688 entries, 0 to 17687
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              17688 non-null  int64  
 1   pred_next90_revenue  17688 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 276.5 KB
None


,user_id,pred_next90_revenue
0,10,0.483388
1,10006,2.426569
2,10007,1.322621
3,1001,1.490462
4,10012,2.183385


In [8]:
df_sgm = pd.read_csv('./segmentation_result.csv')
print(df_sgm.info())
df_sgm.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9370 entries, 0 to 9369
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    9370 non-null   int64  
 1   frequency  9370 non-null   int64  
 2   monetary   9370 non-null   float64
 3   recency    9370 non-null   float64
 4   cluster    9370 non-null   int64  
 5   PCA1       9370 non-null   float64
 6   PCA2       9370 non-null   float64
dtypes: float64(4), int64(3)
memory usage: 512.6 KB
None


,user_id,frequency,monetary,recency,cluster,PCA1,PCA2
0,12631,1,15.560000,0.0,0,-83.344563,-70.558132
1,97874,2,47.805451,0.0,0,-51.086835,-70.414457
2,63363,1,46.530000,0.0,0,-52.379658,-70.420005
3,57946,1,25.490399,0.0,0,-73.415798,-70.513842
4,78411,4,143.972998,0.0,0,45.100055,-69.985830


In [12]:
df = df_sgm.merge(df_pred, how='left', on='user_id')

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9370 entries, 0 to 9369
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              9370 non-null   int64  
 1   frequency            9370 non-null   int64  
 2   monetary             9370 non-null   float64
 3   recency              9370 non-null   float64
 4   cluster              9370 non-null   int64  
 5   PCA1                 9370 non-null   float64
 6   PCA2                 9370 non-null   float64
 7   pred_next90_revenue  9370 non-null   float64
dtypes: float64(5), int64(3)
memory usage: 585.8 KB
None


,user_id,frequency,monetary,recency,cluster,PCA1,PCA2,pred_next90_revenue
0,12631,1,15.560000,0.0,0,-83.344563,-70.558132,3.895203
1,97874,2,47.805451,0.0,0,-51.086835,-70.414457,1.532326
2,63363,1,46.530000,0.0,0,-52.379658,-70.420005,1.518215
3,57946,1,25.490399,0.0,0,-73.415798,-70.513842,5.519188
4,78411,4,143.972998,0.0,0,45.100055,-69.985830,1.738711


In [13]:
# Get the schema of the result
schema = [
    bigquery.SchemaField("user_id", "INTEGER"),
    bigquery.SchemaField("frequency", "INTEGER"),
    bigquery.SchemaField("monetary", "FLOAT"),
    bigquery.SchemaField("recency", "FLOAT"),
    bigquery.SchemaField("cluster", "INTEGER"),
    bigquery.SchemaField("PCA1", "FLOAT"),
    bigquery.SchemaField("PCA2", "FLOAT"),
    bigquery.SchemaField("pred_next90_revenue", "FLOAT"),
]

# Define the new table reference
project_id = credentials.project_id
dataset_id = 'dummy'
table_id = 'cust-potential-thelook'
table_ref = client.dataset(dataset_id, project=project_id).table(table_id)

# Delete the existing destination table if it exists
try:
    client.delete_table(table_ref)
    print(f"Deleted table {table_ref}")
except Exception as e:
    print(f"Table {table_ref} does not exist: {e}")
        
# Create a new table with the schema
table = bigquery.Table(table_ref, schema=schema)
table = client.create_table(table)  # API request


# Load DataFrame to BigQuery table
job = client.load_table_from_dataframe(df, table_ref)

# Wait for the load job to complete
job.result()

print(f"Loaded {job.output_rows} rows into {dataset_id}:{table_id}.")

Deleted table gentle-keyword-423715-j0.dummy.cust-potential-thelook
Loaded 9370 rows into dummy:cust-potential-thelook.
